<a href="https://colab.research.google.com/github/DaeSeokSong/MachineLearningModels/blob/main/Prediction_StockPrice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Import Pakage

In [ ]:
# 기타
import datetime

# AI 학습용
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation

# Funtion

In [ ]:
def DeNormalize(data) :
    returnList = []
    for window in data :
        Normalized_window = [((float(p) * float(window[0])) + (int(p) % int(window[0]) + 1)) for p in window]
        returnList.append(Normalized_window)

    return returnList

# Load Dataset

In [ ]:
filename = '/content/drive/My Drive/DeepLearning/흑우상향/10y_dataset_005930.csv'

data = pd.read_csv(filename)
data.head()

# Cumpute Average Price

In [ ]:
High_prices = data['High'].values
Low_prices = data["Low"].values
Avg_prices = (High_prices + Low_prices) / 2

# Determine Window Size

In [ ]:
# Window size
seq_len = 50
total_seq_len = seq_len + 1 # 예측 기반 데이터 (seq_len)개 + 예측할 데이터 1개

result = []
for idx in range(len(Avg_prices) - total_seq_len) :
    result.append(Avg_prices[idx: idx + total_seq_len])

# Normalize

In [ ]:
Normalized_data = []
for window in result :
    Normalized_window = [((float(p) / float(window[0]) - 1)) for p in window]
    Normalized_data.append(Normalized_window)

result = np.array(Normalized_data)

# 테스트셋 / 훈련셋 분류
row = int(round(result.shape[0] * 0.9))
# 훈련셋 분류 및 랜덤 셔플
train = result[:row, :]
np.random.shuffle(train)

x_train = train[:, :-1] # 총 51개의 윈도우 중 50개
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
y_train = train[:, -1] # 총 51개의 윈도우 중 1개

x_test = result[row:, :-1] # 총 51개의 윈도우 중 50개
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
y_test = result[row:, -1] # 총 51개의 윈도우 중 1개

# Model

In [ ]:
# keras.models 중 모델을 순차적으로 정의하는 클래스
model = Sequential()

# LSTM 모듈 사용, 유닛 50개, 1층
model.add(LSTM(50, return_sequences=True, input_shape=(50,1)))
# LSTM 모듈 사용, 유닛 64개, 2층
model.add(LSTM(64, return_sequences=False))
# Output, 결과 산출
model.add(Dense(1, activation='linear'))
# 손실함수 = mse, Mean Squared Error / Optimizer = rmsprop
model.compile(loss='mse', optimizer='rmsprop')
# 개요 출력
model.summary()

# Training

In [ ]:
# trainSet이나 testSet에 null값 섞여있으면 loss(손실률)이 nan으로 나오니 주의
model.fit(x_train, y_train, # 훈련셋
          validation_data=(x_test, y_test), # 테스트셋, validation_data
          batch_size=10, # 한 번에 몇 개씩 묶어서 학습시킬지, batch_size
          epochs=20) # N번 동안 반복해라, epochs

# Prediction

In [ ]:
# 모델 예측
pred = model.predict(x_test)

fig = plt.figure(facecolor='white')

ax = fig.add_subplot(111)
ax.plot(y_test, label='True')
ax.plot(pred, label='Prediction', linestyle='--')
ax.legend()

plt.show()

# CopyRight

In [ ]:
# 참조 사이트: https://www.learnfit.ai/courses/lstm-stock/lessons/%EB%94%A5%EB%9F%AC%EB%8B%9D-lstm-%EC%89%BD%EA%B2%8C-%EC%9D%B4%ED%95%B4%ED%95%98%EA%B8%B0/